In [ ]:
### 642 homework
policy(s) = if s in (1, 3, 5) a = "E" else a = "SE" end;
function MCsimulation(s, a)
    if s == 6
        R1 = 0
        s1 = 6
    elseif s == 5
        R1 = 10
        s1 = 6
    elseif s in (2, 3, 4)
        R1 = 0
        if a == "E"
            s1 = sample([copy(s), copy(s) + 1], Weights([0.3, 0.7]))
        elseif a in ("NE", "SE")
            s1 = sample([copy(s), copy(s) + 1], Weights([0.85, 0.15]))
        elseif a in ("NW", "SW")
            s1  = sample([copy(s), copy(s) - 1], Weights([0.85, 0.15]))
        elseif a == "W"
            s1 = sample([copy(s), copy(s) - 1], Weights([0.3, 0.7]))
        end
    elseif s == 1
        R1 = 0
        if a in ("NW", "W", "SW")
            s1 = 1
        elseif a in ("NE", "SE")
            s1 = sample([1, 2], Weights([0.85, 0.15]))
        else 
            s1 = sample([1, 2], Weights([0.3, 0.7]))
        end
    end
    return(R1, s1)
end

s = 1;
Vk = Float64[2.5; 3; 5; 6; 10; 0];
γ = 0.9;
for k in 0:100
    k = k + 1
    αk = 1/k
    a = policy(s)
    R1, s1 = MCsimulation(s, a)
    Vk1 = copy(Vk)
    Vk1[s] = Vk[s] + αk * (R1 + γ * Vk[s1] - Vk[s])
    if max(abs.(Vk - Vk1)...) < 0.0001
        break
    end
    Vk = copy(Vk1)
    s = copy(s1)
end
Vk1

In [ ]:
### 642 homework
using LinearAlgebra, StatsBase;
ppi = [1 2 3 4 5 6;
       7 8 9 10 11 12;
       0 0 0.3 0.7 0 0;
       0 0 0 0.85 0.15 0;
       0 0 0 0 0 1;
       0 0 0 0 0 1];
I6 = Matrix(I, 6, 6);
gamma = 0.9;
Rpi = [0; 0; 0; 0; 10; 0];
inv(I6 - gamma * ppi) * Rpi

In [ ]:
#() = missing
A = collect(reshape(1:4, 2, 2))

[A[i, j] for i = 1:size(A, 1), j = 1:size(A, 2) if i > j]

In [239]:
# question 2

using LinearAlgebra, StatsBase;
λ = 10;
μ = 9;
r = 10;
h = 2;
B = 20;
γ = 0.9;
p = λ / (λ + μ);
q = μ / (λ + μ);

#reward(s, a) = push!(pushfirst!(r .* policy.(1:19) .* p .- (1:19) .* h ./ (λ+μ), policy(0) * r), -B * h / (λ+μ));
expected_reward(s, a) = ifelse(s==0, a * r, ifelse(s == B, -B * h / (λ+μ), r * a  - s * h / (λ+μ)));
function MCsimulation(s, a)
    if s == B
        s1 = sample([copy(s), copy(s) - 1], Weights([p, q]))
    elseif s == 0
        s1 = copy(a)
    else 
        s1 = sample([copy(s) - 1, copy(s), copy(s) + 1], Weights([q, (1-a)*p, a*p]))
    end
    reward = expected_reward(s, a)
    return(reward, s1)
end

I21 = Matrix(I, 21, 21)

function pMatrix(policy)
    mat = Matrix(zeros(B + 1, B + 1))
    mat[1, 1] = 1 - policy(0)
    mat[1, 2] = policy(0)
    mat[21, 20] = q
    mat[21, 21] = p
    for i in 2:(B - 1)
        mat[i, i-1] = q
        mat[i, i] = (1 - policy(i - 1)) * p
        mat[i, i+1] = policy(i - 1) * p
    end
    return(mat)
end

function pVec(s, a)
    v = zeros(21)
    if s == 0
        v[1] = copy(1 - a)
        v[2] = copy(a)
    elseif s == B
        v[20] = copy(q)
        v[21] = copy(p)
    else
        v[copy(s)] = copy(q)
        v[copy(s) + 1] = (1 - copy(a)) * copy(p)
        v[copy(s) + 2] = copy(a) * copy(p) 
    end
    return(v)
end



value(policy) = inv(I21 - γ * pMatrix(policy)) * reward(policy);
small_r(a) = ifelse(a == 0, 0, 10)

small_r (generic function with 1 method)

In [164]:
new_action = push!(repeat([0], 20), 0);
c = 0
while c < 100
    c = c + 1
    action = copy(new_action)
    policy(s) = action[s + 1]
    Vk = value(policy)
    old_value = copy(Vk)
    record_action = copy(action)
    for i in 1:20
        running_action = copy(action)
        running_action[copy(i)] = copy(1 - action[copy(i)])
        pold(s) = action[s+1]
        pnew(s) = running_action[s+1]
        vold = small_r.(action) + γ * pMatrix(pold) * Vk
        vnew = small_r.(running_action) + γ * pMatrix(pnew) * Vk
        if max(vold .- vnew ...) <= 0
            action = copy(running_action)
        end
    end

    if max(abs.(action .- new_action) ...) <= 0.1
        break
    end
    new_action = copy(action)
end


In [345]:
using LinearAlgebra, StatsBase;
λ = 10;
μ = 9;
r = 10;
h = 2;
B = 20;
γ = 0.9;
p = λ / (λ + μ);
q = μ / (λ + μ);
expected_reward(s, a) = ifelse(s==0, a * r, ifelse(s == B, -B * h / (λ+μ), r * a  - s * h / (λ+μ)));
function MCsimulation(s, a)
    if s == B
        s1 = sample([copy(s), copy(s) - 1], Weights([p, q]))
    elseif s == 0
        s1 = copy(a)
    else 
        s1 = sample([copy(s) - 1, copy(s), copy(s) + 1], Weights([q, (1-a)*p, a*p]))
    end
    reward = expected_reward(s, a)
    return(reward, s1)
end
ϵ = 0.1
Qk = collect(reshape([push!(ones(20), 0); push!(ones(20), 0)], 21, 2))
s = 0
k = 0
#Qk = []
while k < 10
    k = copy(k + 1)
    αk = copy(1 / k)
    #Qk = copy(Q)
    save_Qk = []
    while s <= 19
        greedy_a = ifelse(Qk[copy(s + 1), 1] > Qk[copy(s + 1), 2], 0, 1)
        explore_a = 1 - copy(greedy_a)
        ak = sample([greedy_a, explore_a], Weights([1 - ϵ, ϵ]))
        observeR, observeS = MCsimulation(copy(s), copy(ak))
        greedy_a1 = ifelse(Qk[copy(observeS + 1), 1] > Qk[copy(observeS + 1), 2], 0, 1)
        explore_a1 = 1 - copy(greedy_a1)
        ak1 = sample([greedy_a1, explore_a1], Weights([1 - ϵ, ϵ]))
        Qk[copy(s) + 1, copy(ak) + 1] = 
        copy(Qk[copy(s) + 1, copy(ak) + 1] + 
                copy(αk) * (copy(observeR) + γ * Qk[copy(observeS) + 1, copy(ak1) + 1] - Qk[copy(s) + 1, copy(ak) + 1]))
        s = copy(observeS)    
    end
    save_Qk = copy(Qk)
    println(max(abs.(Q - save_Qk)...))
    if max(abs.(Q - save_Qk)...) < 0.000001
        break
    end
    #Q = copy(save_Qk)
end


86.90126984248485
0.0


In [340]:
while s <= 19
        greedy_a = ifelse(Qk[copy(s + 1), 1] > Qk[copy(s + 1), 2], 0, 1)
        explore_a = 1 - copy(greedy_a)
        ak = sample([greedy_a, explore_a], Weights([1 - ϵ, ϵ]))
        observeR, observeS = MCsimulation(copy(s), copy(ak))
        greedy_a1 = ifelse(Qk[copy(observeS + 1), 1] > Qk[copy(observeS + 1), 2], 0, 1)
        explore_a1 = 1 - copy(greedy_a1)
        ak1 = sample([greedy_a1, explore_a1], Weights([1 - ϵ, ϵ]))
        Qk[copy(s) + 1, copy(ak) + 1] = 
        copy(Qk[copy(s) + 1, copy(ak) + 1] + 
                copy(αk) * (copy(observeR) + γ * Qk[copy(observeS) + 1, copy(ak1) + 1] - Qk[copy(s) + 1, copy(ak) + 1]))
        s = copy(observeS)    
    end

In [364]:
# yang yangzhuoran 
using LinearAlgebra, StatsBase;
λ = 10;
μ = 9;
r = 10;
h = 2;
B = 20;
γ = 0.9;
p = λ / (λ + μ);
q = μ / (λ + μ);
expected_reward(s, a) = ifelse(s==0, a * r, ifelse(s == B, -B * h / (λ+μ), r * a  - s * h / (λ+μ)));
function MCsimulation(s, a)
    if s == B
        s1 = sample([copy(s), copy(s) - 1], Weights([p, q]))
    elseif s == 0
        s1 = copy(a)
    else 
        s1 = sample([copy(s) - 1, copy(s), copy(s) + 1], Weights([q, (1-a)*p, a*p]))
    end
    reward = expected_reward(s, a)
    return(reward, s1)
end
ϵ = 0.1
Q = collect(reshape([push!(ones(20), 0); push!(ones(20), 0)], 21, 2))
k = 0
while k < 100000
    k = copy(k + 1)
    αk = copy(1 / k)
    Qk = copy(Q)
    s = 0
    while s <= 19
        greedy_a = ifelse(Qk[copy(s + 1), 1] > Qk[copy(s + 1), 2], 0, 1)
        explore_a = 1 - copy(greedy_a)
        ak = sample([greedy_a, explore_a], Weights([1 - ϵ, ϵ]))
        observeR, observeS = MCsimulation(copy(s), copy(ak))
        greedy_a1 = ifelse(Qk[copy(observeS + 1), 1] > Qk[copy(observeS + 1), 2], 0, 1)
        explore_a1 = 1 - copy(greedy_a1)
        ak1 = sample([greedy_a1, explore_a1], Weights([1 - ϵ, ϵ]))
        Qk[copy(s) + 1, copy(ak) + 1] = 
        copy(Qk[copy(s) + 1, copy(ak) + 1] + 
                copy(αk) * (copy(observeR) + γ * Qk[copy(observeS) + 1, copy(ak1) + 1] - Qk[copy(s) + 1, copy(ak) + 1]))
        s = copy(observeS)    
    end
    #println(max(abs.(Q - save_Qk)...))
    if max(abs.(Q - Qk)...) < 0.01
        break
    end
    Q = copy(Qk)
end